<a href="https://colab.research.google.com/github/jivitheshreddy/INFO-5731-Srping2023/blob/main/INFO5731_Assignment_Two_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 1

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter. 

In [ ]:
# Write your code here

import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium import webdriver
url="https://www.imdb.com/title/tt10640346/reviews?ref_=tt_sa_3"
import pandas as pd
import requests

In [ ]:
browser=webdriver.Firefox()
browser.get(url)
html=browser.page_source

In [ ]:
soup=BeautifulSoup(html,'html.parser')

In [ ]:
base_url="https://www.imdb.com/title/tt10640346/reviews?ref_=tt_sa_3"

In [ ]:
master = {}
from tqdm import tqdm

# Define a function to scrape all reviews from a page, including "load more" option
def scrape_reviews(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    review = []
    count = 0
    while True:
        resp = requests.get(url, headers=headers).text
        soup = BeautifulSoup(resp, 'html.parser')
        for i in soup.find_all("div", class_="content"):
            review.append(i.text.strip())
            count += 1
            if count == 10000:
                break
        if count == 10000:
            break
        next_link = soup.find("a", class_="page next")
        if next_link:
            url = next_link['href']
        else:
            break
    table = pd.DataFrame({'Review': review})
    return table

# Loop through all pages and scrape reviews
for i in tqdm(range(1, 1001)):
    url = base_url.format(i)
    master[i] = scrape_reviews(url)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:13<00:00,  1.81it/s]


In [ ]:
big_table=pd.concat([master[i] for i in master],axis=0)

In [ ]:
big_table=big_table.head(10000)

 **Question 2**


(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.


In [ ]:
#Removing Numbers and Puncuation
big_table['Review'] = big_table['Review'].str.replace('[^\w\s]','')
big_table['Review'].head()

C:\Users\jivit\AppData\Local\Temp\ipykernel_20432\14359710.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  big_table['Review'] = big_table['Review'].str.replace('[^\w\s]','')


0    Whether it be orgies showcasing various bodily...
1    So many reviews praise this film for the level...
2    After an interesting opening scene about our m...
3    This film felt like it was written and directe...
4    In the opening scene of BABYLON an elephant on...
Name: Review, dtype: object

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
big_table['Review'] = big_table['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
big_table['Review'].head()

0    Whether orgies showcasing various bodily fluid...
1    So many reviews praise film level debauchery s...
2    After interesting opening scene main character...
3    This film felt like written directed high scho...
4    In opening scene BABYLON elephant pickup empti...
Name: Review, dtype: object

In [ ]:
big_table['Review'] = big_table['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
big_table['Review'].head()

0    whether orgies showcasing various bodily fluid...
1    so many reviews praise film level debauchery s...
2    after interesting opening scene main character...
3    this film felt like written directed high scho...
4    in opening scene babylon elephant pickup empti...
Name: Review, dtype: object

In [ ]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Define the stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define a function to apply the stemmer and lemmatizer to each word in a string
def stem_and_lemmatize(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Stem and lemmatize each word
    stemmed_words = [stemmer.stem(word) for word in words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # Join the stemmed and lemmatized words back into a string
    stemmed_text = ' '.join(stemmed_words)
    lemmatized_text = ' '.join(lemmatized_words)
    # Return the stemmed and lemmatized texts as a tuple
    return (stemmed_text, lemmatized_text)

# Apply the function to each row in the DataFrame
big_table[['stemmed_text', 'lemmatized_text']] = big_table['Review'].apply(stem_and_lemmatize).apply(pd.Series)

# Print the result
print(big_table)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jivit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jivit\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                               Review  \
0   whether orgies showcasing various bodily fluid...   
1   so many reviews praise film level debauchery s...   
2   after interesting opening scene main character...   
3   this film felt like written directed high scho...   
4   in opening scene babylon elephant pickup empti...   
..                                                ...   
20  score 2010my brain hurts wow awful i wouldnt e...   
21  damian chazelle doesnt make bad movies whiplas...   
22  damien chazelle chosen tag along filmmakers de...   
23  a series lengthy chaotic scenes put together s...   
24  i really want understand producers thought who...   

                                         stemmed_text  \
0   whether orgi showcas variou bodili fluid plot ...   
1   so mani review prais film level debaucheri sho...   
2   after interest open scene main charact manni d...   
3   thi film felt like written direct high school ...   
4   in open scene babylon elep

In [ ]:
big_table

,Review,stemmed_text,lemmatized_text
0,whether orgies showcasing various bodily fluid...,whether orgi showcas variou bodili fluid plot ...,whether orgy showcasing various bodily fluid p...
1,so many reviews praise film level debauchery s...,so mani review prais film level debaucheri sho...,so many review praise film level debauchery sh...
2,after interesting opening scene main character...,after interest open scene main charact manni d...,after interesting opening scene main character...
3,this film felt like written directed high scho...,thi film felt like written direct high school ...,this film felt like written directed high scho...
4,in opening scene babylon elephant pickup empti...,in open scene babylon eleph pickup empti bowel...,in opening scene babylon elephant pickup empty...
...,...,...,...
20,score 2010my brain hurts wow awful i wouldnt e...,score 2010mi brain hurt wow aw i wouldnt even ...,score 2010my brain hurt wow awful i wouldnt ev...
21,damian chazelle doesnt make bad movies whiplas...,damian chazel doesnt make bad movi whiplash la...,damian chazelle doesnt make bad movie whiplash...
22,damien chazelle chosen tag along filmmakers de...,damien chazel chosen tag along filmmak decid v...,damien chazelle chosen tag along filmmaker dec...
23,a series lengthy chaotic scenes put together s...,a seri lengthi chaotic scene put togeth substa...,a series lengthy chaotic scene put together su...


In [ ]:
import nltk
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tree import Tree

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def analyze_text(df):
    # Combine all reviews into a single string
    text = ' '.join(big_table['lemmatized_text'])

    # Tokenize the text
    tokens = word_tokenize(text)

    # POS tagging
    pos_tags = pos_tag(tokens)
    noun_count = len([word for word, pos in pos_tags if pos.startswith('N')])
    verb_count = len([word for word, pos in pos_tags if pos.startswith('V')])
    adj_count = len([word for word, pos in pos_tags if pos.startswith('J')])
    adv_count = len([word for word, pos in pos_tags if pos.startswith('R')])

    # Constituency parsing and dependency parsing
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        # Constituency parsing
        tree = nltk.parse.stanford.StanfordParser().parse(nltk.word_tokenize(sentence))
        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                print(subtree)

        # Dependency parsing
        dep_parser = nltk.parse.stanford.StanfordDependencyParser()
        result = dep_parser.raw_parse(sentence)
        dep = result.__next__()
        for triple in dep.triples():
            print(triple)

    # Named entity recognition
    ne_tree = ne_chunk(pos_tags)
    named_entities = {}
    for subtree in ne_tree.subtrees():
        if subtree.label() == 'PERSON':
            entity = ' '.join([word for word, pos in subtree.leaves()])
            named_entities[entity] = named_entities.get(entity, 0) + 1
        elif subtree.label() == 'ORGANIZATION':
            entity = ' '.join([word for word, pos in subtree.leaves()])
            named_entities[entity] = named_entities.get(entity, 0) + 1
        elif subtree.label() == 'GPE':
            entity = ' '.join([word for word, pos in subtree.leaves()])
            named_entities[entity] = named_entities.get(entity, 0) + 1
        elif subtree.label() == 'DATE':
            entity = ' '.join([word for word, pos in subtree.leaves()])
            named_entities[entity] = named_entities.get(entity, 0) + 1

    return noun_count, verb_count, adj_count, adv_count, named_entities


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jivit\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\jivit\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jivit\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
import os
os.environ['STANFORD_PARSER'] = 'C:/stanford-parser/stanford-parser.jar'

In [ ]:
noun_count, verb_count, adj_count, adv_count, named_entities = analyze_text(big_table['lemmatized_text'])
os.environ['STANFORD_MODELS'] = 'C:/stanford-parser/stanford-parser-3.9.2-models.jar'

In [ ]:
import os
os.environ['STANFORD_PARSER'] = 'C:/stanford-parser/stanford-parser.jar'